# Get the dataset

In [ ]:
import numpy as np
import config

dataset_image = np.load(config.x_path)
dataset_label = np.load(config.y_path)
print(dataset_image.shape, dataset_label.shape)
assert(dataset_image.shape[0] == dataset_label.shape[0])


# Boilerplate Code

In [ ]:
from sklearn.model_selection import KFold
import config

kfold = KFold(config.kfold_nsplits, shuffle=True, random_state=config.seed)


In [ ]:
from typing import Tuple
import torch

class Data(torch.utils.data.Dataset):
    def __init__(self, image: np.ndarray, label: np.ndarray) -> None:
        assert(image.shape[0] == label.shape[0])
        self.image = image
        self.label = label
    
    def __len__(self):
        return self.image.shape[0]
    
    def __getitem__(self, idx: int):
        return self.image[idx], self.label[idx]


# Model

In [ ]:
from torch import nn
from torchvision.models import vgg16, VGG16_Weights


def get_model() -> nn.Module:
    model = vgg16(weights=VGG16_Weights.DEFAULT, progress=True)
    in_features = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(in_features, 2)
    return model


In [ ]:
model = get_model()

use_cuda = torch.cuda.is_available()
if not use_cuda:
    print("CUDA not used!")
device = torch.device("cuda" if use_cuda else "cpu")

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

model = model.to(device)
criterion = criterion.to(device)


# Train

In [ ]:
from tqdm import tqdm
def train(train_idx: np.ndarray) -> 'Tuple(float, float)':
    print("TRAINING")
    train = Data(dataset_image[train_idx], dataset_label[train_idx])
    train_dataloader = torch.utils.data.DataLoader(
        train, batch_size=config.batch_size)
    print("MADE DATALOADER")
    total_loss_train = 0
    total_accumulate_train = 0
    for image, label in tqdm(train_dataloader):
        image = image.to(device, dtype=torch.float)
        label = label.to(device, dtype=torch.uint8)
        output = model(image)
        print("OUT: ", output)
        print("LABEL: ", label)

        batch_loss = criterion(output, label)

        total_loss_train += batch_loss

        accumulate = (output.argmax(dim=1) == label).sum()
        total_accumulate_train += accumulate

        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()

    total_loss_train = total_loss_train.item()
    total_accumulate_train = total_accumulate_train.item()
    return (total_loss_train, total_accumulate_train)


In [ ]:
def judge(judge_idx: np.ndarray) -> 'Tuple(float, float)':
    judge = Data(dataset_image[judge_idx], dataset_label[judge_idx])
    judge_dataloader = torch.utils.data.DataLoader(
        judge, batch_size=config.batch_size)

    total_loss_judge = 0
    total_accumulate_judge = 0
    with torch.no_grad():
        for image, label in tqdm(judge_dataloader):
            image = image.to(device, dtype=torch.float)
            label = label.to(device, dtype=torch.uint8)

            output = model(image)

            batch_loss = criterion(output, label)
            total_loss_judge += batch_loss

            accumulate = (output.argmax(dim=1) == label).sum()
            total_accumulate_judge += accumulate

    total_loss_judge = total_loss_judge.item()
    total_accumulate_judge = total_accumulate_judge.item()
    return total_loss_judge, total_accumulate_judge

In [ ]:

min_judge_loss = float('inf')


for epoch, (train_idx, judge_idx) in enumerate(kfold.split(dataset_label)):
    print(epoch)
    total_loss_train, total_accumulate_train = train(train_idx)
    total_loss_judge, total_accumulate_judge = judge(judge_idx)

    print(
        f'''Epochs: {epoch+1} 
        | Train Loss: {total_loss_train / len(train_idx):.3f}
        | Train Accuracy: {total_accumulate_train/len(train_idx):.3f}
        | Val Loss: {total_loss_judge/len(judge_idx):.3f}
        | Val Accuracy: {total_accumulate_judge/len(judge_idx):.3f}'''
        
    )
    if min_judge_loss > total_loss_judge/len(judge_idx):
        min_judge_loss = total_loss_judge/len(judge_idx)
        torch.save(model.state_dict(), "simplemodel.pt")
        print(f"Save model because val loss improve loss {min_judge_loss:.3f}")